In [6]:
import os
import subprocess

os.makedirs("downloads", exist_ok=True)
issues = ["a", "b", "c", "d", "e", "f", "g"]
for issue in issues:
    if not os.path.exists(f"downloads/measure-{issue}.html"):
        subprocess.Popen(["wget", f"https://voterguide.sfelections.org/local-ballot-measures/measure-{issue}", "-O", f"downloads/measure-{issue}.html"])

In [8]:
from openai import OpenAI
client = OpenAI()
debate_instructions="A debate is an organized argument or contest of ideas in which the participants discuss a topic from two opposing sides.  Each side will show in an organized and clever way why they believe to have the right answers.  They will use examples and evidence to support their ideas while working towards a conclusion. The aim of a debate is to convince the opposition that you are right. When the two sides agree on the subject or when one side's arguments are more convincing than the other side that is when the debate comes to a close. "
for issue in issues:
    debater_1 = client.beta.assistants.create(
        name="Debater 1",
        instructions=debate_instructions + ".  You support Measure A. Reference measure-a file and broader context to answer questions.",
        model="gpt-4-1106-preview",
        file_ids=["downloads/measure-a.html"]
    )
    debater_1 = client.beta.assistants.create(
        name="Debater 2",
        instructions=debate_instructions + ".  You oppose Measure A. Reference the included material and broader context to answer questions.",
        model="gpt-4-1106-preview",
        file_ids=["downloads/measure-a.html"]
    )
    show_json(assistant)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable